# Generación de Conjunto de Datos

Este Jupyter Notebook realiza la generación de un conjunto de datos filtrando imágenes basado en la confianza de detección denominada `conf.zoo`. El archivo de entrada es un CSV que contiene información detallada sobre las imágenes, incluyendo el `Path_Final`, `Fecha_Final`, `class.zoo` y `conf.zoo`.

El propósito principal de este Notebook es filtrar y seleccionar únicamente las imágenes que cumplen con ciertos criterios. En particular, se busca incluir en el conjunto de datos final aquellas imágenes cuya confianza de detección `conf.zoo` sea mayor a 0.7. Además, se realiza una clasificación basada en las etiquetas expertas cuando están disponibles.

El flujo de trabajo se describe a continuación:
1. Se carga el archivo CSV de entrada utilizando la biblioteca `pandas` y el método `pd.read_csv()`.
2. Se aplica un filtro para seleccionar las filas que cumplen con las siguientes condiciones:
   - Las imágenes tienen una confianza de detección `conf.zoo` mayor a 0.7.
   - Si la etiqueta experta `class.expert` está disponible, se utiliza esa etiqueta en lugar de `class.zoo`.
3. Se genera un nuevo DataFrame que incluye las columnas `Path_Final`, `Fecha_Final`, `class`, entre otras, utilizando el DataFrame filtrado anteriormente.
4. Se elimina la información innecesaria como `class.expert` y `class.zoo`.
5. El DataFrame resultante se guarda en un nuevo archivo CSV utilizando el método `to_csv()` de `pandas`.

Este proceso permite obtener un conjunto de datos reducido y depurado, listo para su posterior análisis o entrenamiento de modelos de aprendizaje automático.


In [15]:
# Librerias
import pandas as pd

In [16]:
# Constantes

# Ruta de los archivos CSV
csv_path = "../../CSVs/clasificaciones/datiZooRedExp.csv"

umbral_confianza = 0.9

### Leer el CSVs

In [17]:
# Leer el archivo CSV de entrada
df_input = pd.read_csv(csv_path)

# Convertir la columna 'conf.zoo' a tipo numérico
# df_input['conf.zoo'] = pd.to_numeric(df_input['conf.zoo'])

C:\Users\albam\AppData\Local\Temp\ipykernel_12600\1571593799.py:2: DtypeWarning: Columns (0,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_input = pd.read_csv(csv_path)


### Filtros que cumplan las condiciones

In [26]:
# Filtrar las filas que cumplan las condiciones requeridas
df_filtered = df_input[(df_input['class.expert'].notnull()) |
                       (df_input['conf.zoo'] > umbral_confianza)]

# Seleccionar las columnas requeridas y reordenarlas
df_output = df_filtered[['foto_ID.postproceso', 'Station', 'cam', 'date_time', 'class.expert', 'class.zoo']].copy()

# Si class.expert es NA, reemplazarlo por class.zoo
df_output['class'] = df_output.apply(lambda row: row['class.expert'] if pd.notnull(row['class.expert']) else row['class.zoo'], axis=1)

# Eliminar las columnas innecesarias
df_output.drop(['class.expert', 'class.zoo'], axis=1, inplace=True)
df_output = df_output.rename(columns={'foto_ID.postproceso': 'Path_Final'})

### Guardar el DataFrame en un nuevo archivo CSV

In [29]:
# Definir la ruta y el nombre del archivo de salida
output_file = "../../CSVs/c02_images_data_class.csv"

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df_output.to_csv(output_file, index=False)
